In [1]:
# !pip install transformers datasets

In [2]:
# !nvidia-smi

In [3]:
# import os
# from kaggle_secrets import UserSecretsClient
# import wandb

# # Đọc giá trị của WANDB_API_KEY từ secrets
# user_secrets = UserSecretsClient()
# api_key = user_secrets.get_secret("WANDB_API_KEY")

# # Thiết lập giá trị cho biến môi trường WANDB_API_KEY
# os.environ["WANDB_API_KEY"] = api_key

In [4]:
# # Kiểm tra xem API key đã được đặt chưa
# if not api_key:
#     print("Vui lòng đặt WANDB_API_KEY trong môi trường hoặc Kaggle Secrets.")
# else:
#     # Gọi login() mà không cần truyền key
#     wandb.login()


In [5]:
# import wandb
# api_key = "c8ba910e1e95570353acefdb137386065ecae803"
# wandb.login(key=api_key)

In [1]:
from transformers import pipeline, set_seed, GPT2LMHeadModel, AutoTokenizer, TextDataset, AutoModelForCausalLM
import datasets
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np

/home/nampham/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-16 09:14:42.811629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
with open('./data/fashion_15_10_2022_train.txt', 'r') as file:
    line_count = 0
    for line in file:
        print(line.strip())
        line_count += 1
        
        # Nếu đã đọc đủ 10 dòng, thoát khỏi vòng lặp
        if line_count == 10:
            break


áo khoác<|beginofdes|>cách phối đồ với giày vans nam đã phá bỏ đi sự cứng nhắc của giày tây. chiếc áo biker jacket – loại áo khoác da đặc trưng của nhưng tay đua dân chơi của những anh chàng cá tính mạnh đã được kết hợp khéo léo với giày vans. với nhiều ý tưởng thay đổi từ kiểu dáng cũng như phong cách khác nhau và dựa vào tâm lí khách hàng mà nhà sản xuất đã đưa ra rất nhiều mẫu áo khoác biker jacket khác nhau với các tông màu đặc biệt. nếu các bạn nam lựa chọn set đồ này thì màu đen luôn là màu lí tưởng và đạt được sự lựa chọn nhiều nhất.<|endofdes|>
xxxxxEndxxxxx
trang phục<|beginofdes|>thông thường, khi nam giới sở hữu một vóc dáng nhỏ bé và chiều cao hạn chế thì khó trong việc lựa chọn trang phục. do vậy, việc mix đồ phải được thực hiện cẩn thận và chăm chút hơn.<|endofdes|>
xxxxxEndxxxxx
cách mix đồ<|beginofdes|>với những cô nàng tuổi 30 thì nên mặc gì trong những buổi hẹn hò? quyến rũ, gợi cảm hay chọn thanh lịch, dịu dàng? dù bạn thích phong cách gì thì cũng nên chọn những set 

In [8]:
with open('./data/fashion_15_10_2022_test.txt', 'r') as file:
    line_count = 0
    for line in file:
        print(line.strip())
        line_count += 1
        
        # Nếu đã đọc đủ 10 dòng, thoát khỏi vòng lặp
        if line_count == 10:
            break

trang phục<|beginofdes|>trang phục với họa tiết hoa là item cần có trong tủ quần áo ngày hè của phái nữ. tuy nhiên, xu hướng này đều thay đổi theo từng năm. mùa hè năm nay, những chiếc đầm hoa tay bồng hoặc cổ chữ v chính là lựa chọn số 1 của các mỹ nhân việt cho những chuyến du lịch hoặc kết hợp với streetstyle, phong cách hàng ngày.<|endofdes|>
xxxxxEndxxxxx
đồ lót<|beginofdes|>kiểu quần nội y đầu tiên được gọi tên chính là quần bikini. bikini là mẫu đồ lót cao cấp phổ biến nhất, có độ dài vừa phải ôm trọn vòng ba. thường được làm từ các chất liệu vải thoáng mát. vì vậy, bikini rất thích hợp để mặc hàng ngày và đặc biệt trong những ngày “đèn đỏ" vì sự thoải mái mà chúng mang lại<|endofdes|>
xxxxxEndxxxxx
áo len<|beginofdes|>set đồ cuối cùng mà [Tên shop] muốn chia sẻ đến chị em là váy suông hoa nhí và quần ống rộng mix áo len ôm. màu sắc được chọn lựa cho 2 set đồ này là màu xanh rêu pha chút vàng cỏ úa, giúp set đồ của bạn có sự khác biệt và nổi bật 

In [10]:
model = AutoModelForCausalLM.from_pretrained("./models")
tokenizer = AutoTokenizer.from_pretrained("./models")
special_tokens_dict = {'additional_special_tokens': ['<|beginofdes|>','<|endofdes|>', '<br>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def get_text_ds(path):
    lines = []
    text = ''
    with open(path) as f:
        for line in f:
            if line.find('xxxxxEndxxxxx') != -1:
                if text.strip():
                    lines.append(text.strip())
                    text = ''
                else:
                    continue
            else:
                text = text + line
    vi_ds = datasets.Dataset.from_dict({'text': lines})
    return vi_ds

In [12]:
train_ds = get_text_ds('./data/fashion_15_10_2022_train.txt')
test_ds = get_text_ds('./data/fashion_15_10_2022_test.txt')

In [13]:
train_ds , test_ds

(Dataset({
     features: ['text'],
     num_rows: 40803
 }),
 Dataset({
     features: ['text'],
     num_rows: 10201
 }))

In [14]:
train_ds[0]

{'text': 'áo khoác<|beginofdes|>cách phối đồ với giày vans nam đã phá bỏ đi sự cứng nhắc của giày tây. chiếc áo biker jacket – loại áo khoác da đặc trưng của nhưng tay đua dân chơi của những anh chàng cá tính mạnh đã được kết hợp khéo léo với giày vans. với nhiều ý tưởng thay đổi từ kiểu dáng cũng như phong cách khác nhau và dựa vào tâm lí khách hàng mà nhà sản xuất đã đưa ra rất nhiều mẫu áo khoác biker jacket khác nhau với các tông màu đặc biệt. nếu các bạn nam lựa chọn set đồ này thì màu đen luôn là màu lí tưởng và đạt được sự lựa chọn nhiều nhất.<|endofdes|>'}

In [15]:
test_ds[0]

{'text': 'trang phục<|beginofdes|>trang phục với họa tiết hoa là item cần có trong tủ quần áo ngày hè của phái nữ. tuy nhiên, xu hướng này đều thay đổi theo từng năm. mùa hè năm nay, những chiếc đầm hoa tay bồng hoặc cổ chữ v chính là lựa chọn số 1 của các mỹ nhân việt cho những chuyến du lịch hoặc kết hợp với streetstyle, phong cách hàng ngày.<|endofdes|>'}

In [16]:
context_length = 512
def tokenize(batch):
  outputs = tokenizer(
      batch['text'],
      truncation=True, # Cắt ngắn các câu dài thành max_length
      max_length=context_length, # Định nghĩa độ dài tối đa cho phép 
      return_overflowing_tokens=True, # Trả về token vượt quá max_length
      return_length=True # Trả về độ dài chuỗi được phân tách từ 
      #padding='max_length'
  )
  input_batch = []
  for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      input_batch.append(input_ids)
  result = {}
  result['input_ids'] = input_batch
  # result["labels"] = result["input_ids"].copy()
  return result


# Sử dụng tokenize để tiền xử lý dữ liệu văn bản trong tập train_ds
tokenized_train_datasets = train_ds.map(
    tokenize, batched=True, remove_columns=['text']

)

Map: 100%|██████████| 40803/40803 [00:05<00:00, 7651.43 examples/s]


Ví dụ : 
batch = {
"text" : ["Tôi là một người Việt Nam.", "Tôi sống ở Hà Nội.","Tôi thích học Python."
}

tokenize() : tạo ra đầu vào mới 

input_batch = [[101, 102, 103, 104, 105, 106], [101, 102, 103, 104, 105], [101, 102, 103, 104, 105, 106, 107, 108]]

Từ điển kết quả sẽ là :
result = {
  "input_ids": input_batch
}


In [17]:
tokenized_test_datasets = test_ds.map(
    tokenize, batched=True, remove_columns=['text']
)

Map: 100%|██████████| 10201/10201 [00:01<00:00, 8129.11 examples/s]


In [18]:
train_data_path = 'data/train'
tokenized_train_datasets.save_to_disk(train_data_path)

test_data_path = 'data/test'
tokenized_test_datasets.save_to_disk(test_data_path)

Saving the dataset (1/1 shards): 100%|██████████| 10201/10201 [00:00<00:00, 629225.79 examples/s]


In [19]:
tokenized_train_datasets[0]

{'input_ids': [463,
  4141,
  50259,
  8489,
  1381,
  1317,
  379,
  3511,
  270,
  3983,
  1644,
  378,
  1536,
  1113,
  648,
  546,
  3128,
  2225,
  338,
  3511,
  3179,
  18,
  1143,
  1686,
  22740,
  27202,
  1138,
  1014,
  1686,
  4141,
  1770,
  1081,
  2622,
  338,
  661,
  1083,
  2220,
  596,
  1391,
  338,
  445,
  764,
  2499,
  1095,
  899,
  887,
  378,
  371,
  737,
  615,
  4123,
  5162,
  379,
  3511,
  270,
  3983,
  18,
  379,
  511,
  860,
  1578,
  1195,
  955,
  458,
  2371,
  2549,
  509,
  438,
  1476,
  776,
  792,
  1088,
  314,
  2555,
  494,
  802,
  4003,
  875,
  572,
  638,
  530,
  654,
  688,
  378,
  897,
  471,
  712,
  511,
  1488,
  1686,
  4141,
  22740,
  27202,
  792,
  1088,
  379,
  349,
  3015,
  1522,
  1081,
  986,
  18,
  1164,
  349,
  810,
  1644,
  1902,
  1273,
  6862,
  1317,
  452,
  662,
  1522,
  2198,
  1298,
  341,
  1522,
  4003,
  1578,
  314,
  1089,
  371,
  546,
  1902,
  1273,
  511,
  617,
  18,
  50258]}

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, EarlyStoppingCallback
import datasets
import numpy as np
import math

from datasets import load_from_disk
import torch


# Load dataset 
train_dataset = load_from_disk("./data/train")
test_dataset = load_from_disk("./data/test")

In [2]:
from transformers import pipeline, set_seed, GPT2LMHeadModel, AutoTokenizer, TextDataset, AutoModelForCausalLM
import datasets
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
import time 
import torch 
import torch.nn.functional as F

In [3]:
model = AutoModelForCausalLM.from_pretrained("./models")
tokenizer = AutoTokenizer.from_pretrained("./models")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def generate(model, input_ids, max_length=100, k = 3, on = 3):
    generated_sequence = model.generate(
        input_ids= input_ids,
        max_new_tokens=max_length,
        num_beams=k,
        no_repeat_ngram_size=3,
        num_return_sequences=3,
        eos_token_id=50259,
        use_cache=True
    )
    
    for i in range(on):
        x = generated_sequence[i]
#         print('---',x)
        text = tokenizer.decode(
            x,
            skip_special_tokens=False,
            clean_up_tokenization_spaces=True,
        )
        print(text)

In [5]:
# def generate(model, input_ids, max_length=100, k = 3, on = 3):
#     # Cấu hình lại beam search để không lấy tổng hòa xác suất
#     model.config.alpha = 0

#     # Tạo ra 3 chuỗi câu khác nhau
#     generated_sequences = model.generate(
#         input_ids=input_ids,
#         max_new_tokens=max_length,
#         num_beams=k,
#         num_return_sequences=on,
#         eos_token_id=50259,
#         use_cache=True
#     )

#     # In ra 3 chuỗi câu
#     for i in range(on):
#         x = generated_sequences[i]
#         print('---', x)
#         text = tokenizer.decode(
#             x,
#             skip_special_tokens=False,
#             clean_up_tokenization_spaces=True,
#         )
#         print(text)


In [6]:
tt = 'quần lửng<|beginofdes|>'
input_ids = tokenizer("{}".format(tt),return_tensors='pt').input_ids
generate(model,input_ids,200,3,3)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50259 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


quần lửng<|beginofdes|>với những cô nàng có chiều cao khiêm tốn thì hãy chọn những chiếc quần lửng có chiều dài trên mắt cá chân một chút để giúp đôi chân trông dài hơn. còn đối với những bạn gái cao ráo thì có thể chọn những mẫu quần lửng ống rộng để mix cùng áo sơ mi.<|endofdes|>quần lửng sẽ giúp bạn trông cao hơn và che đi khuyết điểm đôi chân của bạn.<|endofdes|>bạn có thể kết hợp cùng với một đôi giày cao gót để có một set đồ hoàn hảo nhất nhé.<|endofdes|>với những bạn nữ có thân hình hơi mũm mĩm một chút thì đừng ngần ngại chọn cho mình chiếc quần cạp cao. nó sẽ giúp cho bạn trông thon gọn và cao hơn đó.<|endofdes|>bên cạnh đó, bạn cũng có thể phối cùng với những đôi giày đế bằng để có thể tôn lên đôi chân dài của mình.<|endofdes|>nếu như bạn là một cô nàng nấm lùn thì đừng quên kết hợp với những chiếc thắt lưng để giúp cho vòng eo của bạn trông nhỏ hơn.<|endofdes|> bạn có
quần lửng<|beginofdes|>với những cô nàng có chiều cao khiêm tốn thì hãy chọn những chiếc quần lửng có chiều 

In [25]:
!pip install rank_bm25

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
# from rank_bm25 import BM25Okapi

# # Đọc dữ liệu từ file txt
# with open('/kaggle/input/fashion7/fashion_15_10_2022_train.txt', 'r', encoding='utf-8') as file:
#     documents = [line.strip() for line in file]

# # Tiền xử lý dữ liệu (bạn có thể thêm các bước tiền xử lý khác)
# preprocessed_documents = [document.lower().split() for document in documents]

# # Tạo mô hình BM25
# bm25 = BM25Okapi(preprocessed_documents)

# # Đặt truy vấn (query)
# query = "Quần âu".lower().split()

# # Tính điểm tương đồng
# scores = bm25.get_scores(query)

# # Xếp hạng các tài liệu
# ranked_documents = sorted(zip(range(len(documents)), scores), key=lambda x: x[1], reverse=True)[:3]

# # Hiển thị nội dung các tài liệu được xếp hạng
# for idx, score in ranked_documents:
#     print(f"Document {idx + 1}: Score = {score}")
#     print(f"Content: {documents[idx]}\n")


from rank_bm25 import BM25Okapi

# Đọc dữ liệu từ file train.txt
with open('/kaggle/input/fashion7/fashion_15_10_2022_train.txt', 'r', encoding='utf-8') as train_file:
    train_documents = [line.strip() for line in train_file]

# Đọc dữ liệu từ file test.txt
with open('/kaggle/input/fashion7/fashion_15_10_2022_test.txt', 'r', encoding='utf-8') as test_file:
    test_documents = [line.strip() for line in test_file]

# Gộp cả hai danh sách tài liệu
all_documents = train_documents + test_documents

# Tiền xử lý dữ liệu
preprocessed_documents = [document.lower().split() for document in all_documents]

# Tạo mô hình BM25
bm25 = BM25Okapi(preprocessed_documents)

# Đặt truy vấn (query)
query = "Quần âu".lower().split()

# Tính điểm tương đồng
scores = bm25.get_scores(query)

# Xếp hạng các tài liệu
ranked_documents = sorted(zip(range(len(all_documents)), scores), key=lambda x: x[1], reverse=True)[:3]

# Hiển thị nội dung các tài liệu được xếp hạng
for idx, score in ranked_documents:
    print(f"Document {idx + 1}: Score = {score}")
    print(f"Content: {all_documents[idx]}\n")



Document 94835: Score = 9.751437243656115
Content: quần jeans<|beginofdes|>so với quần jeans, quần âu mang đến sự thanh lịch, lịch lãm đúng chuẩn đàn ông. quần âu chính là mảnh ghép tuyệt vời cho các chàng trai trong set đồ mùa noel đấy.<|endofdes|>

Document 31873: Score = 9.736146858171102
Content: quần âu<|beginofdes|>trong tủ đồ của nam giới chắc chắn sẽ không thể thiếu đi những chiếc quần âu lịch lãm. những chàng trai hàn quốc luôn chọn những chiếc quần âu nhẹ nhàng lịch lãm. kết hợp quần âu cùng áo sơ mi tone sáng mang đến một style chuẩn "nam thần" luôn.<|endofdes|>

Document 22201: Score = 9.666548428206983
Content: quần âu<|beginofdes|>cách dễ dàng nhất để có được phong cách xuân/hè trẻ trung, sành điệu và thanh lịch là nên đầu tư cho những chiếc quần như jeans, quần âu ống đứng và quần âu ống rộng.<|endofdes|>



In [27]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = AutoModelForCausalLM.from_pretrained("/kaggle/input/best-model-1-training-large-model/training_article/best_model")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/best-model-1-training-large-model/training_article/best_model")

# Your input sentence
input_sentence = "so với quần jeans, quần âu mang đến sự thanh lịch, lịch lãm đúng chuẩn đàn ông. quần âu chính là mảnh ghép tuyệt vời cho các chàng trai trong set đồ mùa noel đấy"

# Tokenize input sentence
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

# Generate more text using GPT-2
output = model.generate(input_ids, max_length=260, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode and display the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


so với quần jeans, quần âu mang đến sự thanh lịch, lịch lãm đúng chuẩn đàn ông. quần âu chính là mảnh ghép tuyệt vời cho các chàng trai trong set đồ mùa noel đấy.quần âu có thể kết hợp với nhiều kiểu áo khác nhau, từ áo sơ mi, áo thun, đến áo len, sơ vin hay bỏ áo ngoài quần cũng đều đẹp. bạn có biết cách phối đồ với áo polo nam đẹp không? hãy cùng [Tên shop] tìm hiểu ngay nhé!áo polo có cổ màu trắng, mix cùng quần tây màu đen, thêm một đôi giày thể thao màu xám. một chiếc mũ lưỡi trai màu xanh rêu, một túi đeo chéo màu nâu. thêm chiếc áo khoác kaki màu be, màu kem. tất cả tạo nên một set trang phục hoàn hảo cho ngày noel. set này sẽ giúp bạn trở nên thật phong cách và thời trang. nếu bạn muốn thay đổi một chút cho set quần áo mùa đông của mình, hãy thử ngay set áo này nhé. với một sự kết đôinh nhẹ nhàng, đơn giản nhưng lại không hề đơn điệu đâu nhé các anh chàng. các bạn hãy chọn cho mình một bộ trang phúc noel thật đẹp và phù hợp nhé phối quần kaki nam với giày tây nam màu ghi xám áo

In [28]:
# from rank_bm25 import BM25Okapi
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

# # Đọc dữ liệu từ file txt
# with open('/kaggle/input/fashion7/fashion_15_10_2022_train.txt', 'r', encoding='utf-8') as file:
#     documents = [line.strip() for line in file]

# # Tiền xử lý dữ liệu (bạn có thể thêm các bước tiền xử lý khác)
# preprocessed_documents = [document.lower().split() for document in documents]

# # Tạo mô hình BM25
# bm25 = BM25Okapi(preprocessed_documents)

# # Đặt truy vấn (query)
# query = "Quần âu".lower().split()

# # Tính điểm tương đồng
# scores = bm25.get_scores(query)

# # Xếp hạng các tài liệu
# ranked_documents = sorted(zip(range(len(documents)), scores), key=lambda x: x[1], reverse=True)[:3]

# # Your GPT-2 model and tokenizer initialization
# model = GPT2LMHeadModel.from_pretrained("/kaggle/input/best-model-1-training-large-model/training_article/best_model")
# tokenizer = GPT2Tokenizer.from_pretrained("/kaggle/input/best-model-1-training-large-model/training_article/best_model")

# # Choose the document you want to use as input for GPT-2
# selected_document_index = ranked_documents[0][0]  # You can change this index as needed
# selected_document_content = documents[selected_document_index]

# # Tokenize input sentence
# input_ids = tokenizer.encode(selected_document_content, return_tensors='pt')

# # Generate more text using GPT-2
# output = model.generate(input_ids, max_length=260, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# # Decode and display the generated text
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(f"Generated Text:\n{generated_text}")





In [29]:
from rank_bm25 import BM25Okapi
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_from_disk

# Load dataset
train_dataset = load_from_disk("article_512/train")
test_dataset = load_from_disk("article_512/test")

In [30]:
# Extract 'input_ids' from the training dataset
train_documents = train_dataset['input_ids']

In [31]:
# Extract 'input_ids' from the test dataset (optional)
test_documents = test_dataset['input_ids']

In [32]:
# train_documents , test_documents

In [33]:
# Combine train and test documents (if needed)
all_documents = train_documents + test_documents

In [34]:
# Preprocess the documents
preprocessed_documents = [document for document in all_documents]

In [35]:
# Create BM25 model
bm25 = BM25Okapi(preprocessed_documents)

# Set query
query = "Quần âu".lower().split()

# Calculate similarity scores
scores = bm25.get_scores(query)

# Rank the documents
ranked_documents = sorted(zip(range(len(all_documents)), scores), key=lambda x: x[1], reverse=True)[:3]

# Initialize GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/kaggle/input/best-model-1-training-large-model/training_article/best_model")
tokenizer = GPT2Tokenizer.from_pretrained("/kaggle/input/best-model-1-training-large-model/training_article/best_model")

# Choose the document for GPT-2 input
selected_document_index = ranked_documents[0][0]
selected_document_content = all_documents[selected_document_index]

# Convert to string if necessary
selected_document_content_str = tokenizer.decode(selected_document_content, skip_special_tokens=True)

# Tokenize input sentence
input_ids = tokenizer.encode(selected_document_content_str, return_tensors='pt')

# Generate more text using GPT-2
output = model.generate(input_ids, max_length=260, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode and display the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Generated Text:\n{generated_text}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
áo khoáccách phối đồ với giày vans nam đã phá bỏ đi sự cứng nhắc của giày tây. chiếc áo biker jacket – loại áo khoác da đặc trưng của nhưng tay đua dân chơi của những anh chàng cá tính mạnh đã được kết hợp khéo léo với giày vans. với nhiều ý tưởng thay đổi từ kiểu dáng cũng như phong cách khác nhau và dựa vào tâm lí khách hàng mà nhà sản xuất đã đưa ra rất nhiều mẫu áo khoác biker jacket khác nhau với các tông màu đặc biệt. nếu các bạn nam lựa chọn set đồ này thì màu đen luôn là màu lí tưởng và đạt được sự lựa chọn nhiều nhất.áo biker là một trong những item thời trang nam được phái mạnh yêu thích và lựa chon nhiều. vậy nên không có lí do gì để bạn không lựa cho mình một chiếc biker trong tủ đồ của mình. hãy cùng [Tên shop] khám phá những cách phối áo vons với biker nhé. cùng xem nhé!nam phối với áo phông đen dụ phối với quần jean đen, áo sơ mi đen và giày đen. thêm một đôi giày thể thao màu trắng nữa là hoàn hảo. bạn có thể đi cùng đôi tất dài màu xanh quân đội hoặc mộ

In [36]:
selected_document_content_str

'áo khoáccách phối đồ với giày vans nam đã phá bỏ đi sự cứng nhắc của giày tây. chiếc áo biker jacket – loại áo khoác da đặc trưng của nhưng tay đua dân chơi của những anh chàng cá tính mạnh đã được kết hợp khéo léo với giày vans. với nhiều ý tưởng thay đổi từ kiểu dáng cũng như phong cách khác nhau và dựa vào tâm lí khách hàng mà nhà sản xuất đã đưa ra rất nhiều mẫu áo khoác biker jacket khác nhau với các tông màu đặc biệt. nếu các bạn nam lựa chọn set đồ này thì màu đen luôn là màu lí tưởng và đạt được sự lựa chọn nhiều nhất.'

In [ ]:
import requests

# Define the URL
url = "http://127.0.0.1:8000/generate/"

# Input data
data = {
    "prompt": "quần âu nam<|beginofdes|>",
    "max_length": 100,  # optional, adjust as needed
    "k": 1,  # optional, adjust as needed
    "on": 1  # optional, adjust as needed
}

# Make a POST request
response = requests.post(url, json=data)

# Print the response
print(response.json())